<a href="https://colab.research.google.com/github/linyuehzzz/5526_neural_networks/blob/master/rbf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Lab 2**
This code implements an RBF network.  
Yue Lin (lin.3326 at osu.edu)  
Created: 10/3/2020